# Lab 04-1. Multivariate Linear Regression

- 지금까지 다룬 선형회귀가 1개의 정보를 통해 1개의 추측값을 구하는 Simple Linear Regression이라면,
- 오늘은 여러개, 복수의 정보를 통해 1개의 추측값을 구하는 Mulitvariate Linear Rregression을 알아보자.

- 입력변수가 3개, 4개까지야 뭐 y = W1*x1 + W2*x2 + W3*x3 + b 같은 걸로 정의할 수 있지만,
- 입력변수가 100개, 200개, 300개까지 다양한 dataset들로 늘어난다면...?
- 그걸 해결하는게 Matrix, matmul함수를 통해 할 수 있다.
- 행렬곱을 이용해 x를 하나만 쓰고도 matmul행렬곱 함수를 이용해 각 vector의 element마다 W를 곱해주는 것으로 식도 간단하게 만들고, 연산도 간단하게 할 수 있게 해결!

In [3]:
import torch

# 데이터
x_train = torch.FloatTensor([[73, 80, 85],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

In [4]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([70.7484, 81.4051, 80.1995, 87.3474, 62.0861]) Cost: 9097.098633
Epoch    2/20 hypothesis: tensor([109.9108, 126.4677, 124.5944, 135.6989,  96.4548]) Cost: 2795.581299
Epoch    3/20 hypothesis: tensor([131.5885, 151.4128, 149.1694, 162.4640, 115.4803]) Cost: 864.642395
Epoch    4/20 hypothesis: tensor([143.5875, 165.2216, 162.7731, 177.2798, 126.0126]) Cost: 272.952332
Epoch    5/20 hypothesis: tensor([150.2288, 172.8659, 170.3035, 185.4811, 131.8434]) Cost: 91.639618
Epoch    6/20 hypothesis: tensor([153.9043, 177.0979, 174.4720, 190.0208, 135.0716]) Cost: 36.076332
Epoch    7/20 hypothesis: tensor([155.9382, 179.4409, 176.7796, 192.5337, 136.8593]) Cost: 19.045399
Epoch    8/20 hypothesis: tensor([157.0632, 180.7382, 178.0569, 193.9245, 137.8493]) Cost: 13.821927
Epoch    9/20 hypothesis: tensor([157.6851, 181.4567, 178.7641, 194.6944, 138.3980]) Cost: 12.216333
Epoch   10/20 hy

## nn.Module
- 인공신경망, 즉 hypothesis를 간편하게 쓰기 위한 pytorch 모듈
- nn.Module을 상속해서 hyopthesis로 쓸 model 클래스 선언할 수 있음(아래 예시 참고)
- nn.Linear(3, 1)
-  입력 차원 : 3
-  출력 차원 : 1
- hypothesis계산은 forward()에서!
- Gradient 계산은 PyTorch가 알아서 해준다 backward()

## F.mse_loss
- torch.nn.functional에서 제공하는 loss function 사용
- 쉽게 다른 loss와 교체 가능!(l1.loss, smooth_l1_loss 등...)
- cost 계산이 간편해짐

In [7]:
import torch.nn as nn
import torch.nn.functional as F

In [11]:
# nn.Module을 사용한 MultivariateLinearRegressionModel 클래스 선언
# F.mse_loss
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        return self.linear(x)


# 데이터
x_train = torch.FloatTensor([[73, 80, 85],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
# W = torch.zeros((3, 1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # hypothesis = x_train.matmul(W) + b # or .mm or @
    hypothesis = model(x_train)
    
    # cost 계산
    # cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    1/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    2/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    3/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    4/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    5/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    6/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    7/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    8/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch    9/20 hypothesis: tensor([48.1312, 59.2521, 57.7312, 62.1733, 46.2006]) Cost: 13727.607422
Epoch   10